# **Part 1: Run MobileNet on GPU**

In this tutorial, we will explore how to train a neural network with PyTorch.

### Setup (5%)

We will first install a few packages that will be used in this tutorial and also define the path of CUDA library:

In [1]:
!pip install torchprofile 1>/dev/null
!ldconfig /usr/lib64-nvidia 2>/dev/null
!pip install onnx 1>/dev/null
!pip install onnxruntime 1>/dev/null

We will then import a few libraries:

In [28]:
import random

import numpy as np
import torch
import torchvision
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
from torchprofile import profile_macs
from torchvision.datasets import *
from torchvision.transforms import *
from tqdm.auto import tqdm

In [29]:
print(torch.__version__)
print(torchvision.__version__)

2.5.1+cu124
0.20.1+cu124


To ensure the reproducibility, we will control the seed of random generators:

In [30]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

We must decide the HYPER-parameter before training the model:

In [31]:
NUM_CLASSES = 10

# TODO:
# Decide your own hyper-parameters
BATCH_SIZE = 128
LEARNING_RATE = 0.00002
NUM_EPOCH = 10

### Data  (5%)

In this lab, we will use CIFAR-10 as our target dataset. This dataset contains images from 10 classes, where each image is of
size 3x32x32, i.e. 3-channel color images of 32x32 pixels in size.

Before using the data as input, we can do data pre-processing with transform function:

In [32]:
# TODO:
# Resize images to 224x224, i.e., the input image size of MobileNet,
# Convert images to PyTorch tensors, and
# Normalize the images with mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
transform = torchvision.transforms.Compose([
  torchvision.transforms.Resize((224, 224)),
  torchvision.transforms.ToTensor(),
  torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


dataset = {}
for split in ["train", "test"]:
  dataset[split] = CIFAR10(
    root="data/cifar10",
    train=(split == "train"),
    download=True,
    transform=transform,
  )

100%|██████████| 170M/170M [00:05<00:00, 31.1MB/s]


Extracting data/cifar10/cifar-10-python.tar.gz to data/cifar10
Files already downloaded and verified


To train a neural network, we will need to feed data in batches.

We create data loaders with the batch size determined previously in setup section:

In [33]:
dataflow = {}
for split in ['train', 'test']:
  dataflow[split] = DataLoader(
    dataset[split],
    batch_size=BATCH_SIZE,
    shuffle=(split == 'train'),
    num_workers=0,
    pin_memory=True,
    drop_last=True
  )

We can print the data type and shape from the training data loader:

In [34]:
for inputs, targets in dataflow["train"]:
  print(f"[inputs] dtype: {inputs.dtype}, shape: {inputs.shape}")
  print(f"[targets] dtype: {targets.dtype}, shape: {targets.shape}")
  break

[inputs] dtype: torch.float32, shape: torch.Size([128, 3, 224, 224])
[targets] dtype: torch.int64, shape: torch.Size([128])


### Model (10%)

In this tutorial, we will import MobileNet provided by torchvision, and use the pre-trained weight:

In [35]:
# TODO:
# Load pre-trained MobileNetV2
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights
model = mobilenet_v2(weights = MobileNet_V2_Weights)
print(model)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 131MB/s]

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

You should observe that the output dimension of the classifier does not match the number of cleasses in CIFAR-10.

Now change the output dimension of the classifer to number of classes:

In [36]:
# TODO:
# Change the output dimension of the classifer to number of classes
model.classifier[1] = nn.Linear(model.classifier[1].in_features, NUM_CLASSES)
print(model)

# Send the model from cpu to gpu
model = model.to("cuda")

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

Now the output dimension of the classifer matches.

As this course focuses on efficiency, we will then inspect its model size and (theoretical) computation cost.


* The model size can be estimated by the number of trainable parameters:

In [37]:
num_params = 0
for param in model.parameters():
  if param.requires_grad:
    num_params += param.numel()
print("#Params:", num_params)

#Params: 2236682


* The computation cost can be estimated by the number of [multiply–accumulate operations (MACs)](https://en.wikipedia.org/wiki/Multiply–accumulate_operation) using [TorchProfile](https://github.com/zhijian-liu/torchprofile), we will further use this profiling tool in the future labs .

In [38]:
num_macs = profile_macs(model, torch.zeros(1, 3, 224, 224).cuda())
print("#MACs:", num_macs)

#MACs: 306186464


This model has 2.2M parameters and requires 306M MACs for inference. We will work together in the next few labs to improve its efficiency.

### Optimization (10%)

As we are working on a classification problem, we will apply [cross entropy](https://en.wikipedia.org/wiki/Cross_entropy) as our loss function to optimize the model:

In [39]:
# TODO:
# Apply cross entropy as our loss function
criterion = nn.CrossEntropyLoss()

We should decide an optimizer for the model:

In [40]:
# TODO:
# Choose an optimizer.
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)

(Optional) We can apply a learning rate scheduler during the training:

In [41]:
# TODO(optional):
# scheduler = StepLR(optimizer, step_size=10, gamma=0.5)

### Training (25%)

We first define the function that optimizes the model for one batch:

In [42]:
def train_one_batch(
  model: nn.Module,
  criterion: nn.Module,
  optimizer: Optimizer,
  inputs: torch.Tensor,
  targets: torch.Tensor,
  scheduler
) -> None:

    # TODO:
    # Step 1: Reset the gradients (from the last iteration)
    optimizer.zero_grad()

    # Step 2: Forward inference
    out = model(inputs)

    # Step 3: Calculate the loss
    loss = criterion(out, targets)

    # Step 4: Backward propagation
    loss.backward()

    # Step 5: Update optimizer
    optimizer.step()

    # (Optional Step 6: scheduler)
    # scheduler.step()

    return loss.item()


We then define the training function:

In [43]:
def train(
    model: nn.Module,
    dataflow: DataLoader,
    criterion: nn.Module,
    optimizer: Optimizer,
    scheduler: LRScheduler
):

  model.train()
  losses = []

  for inputs, targets in tqdm(dataflow, desc='train', leave=False):
    # Move the data from CPU to GPU
    inputs = inputs.cuda()
    targets = targets.cuda()

    # Call train_one_batch function
    loss = train_one_batch(model, criterion, optimizer, inputs, targets, scheduler)
    losses.append(loss)
  return np.mean(losses)

Last, we define the evaluation function:

In [44]:
def evaluate(
  model: nn.Module,
  dataflow: DataLoader
) -> float:

    model.eval()
    num_samples = 0
    num_correct = 0

    with torch.no_grad():
        for inputs, targets in tqdm(dataflow, desc="eval", leave=False):
            # TODO:
            # Step 1: Move the data from CPU to GPU
            inputs = inputs.cuda()

            # Step 2: Forward inference
            out = model(inputs)

            # Step 3: Convert logits to class indices (predicted class)
            predicts = out.argmax(dim=1).cpu()

            # Update metrics
            num_samples += targets.size(0)
            num_correct += (predicts == targets).sum()

    return (num_correct / num_samples * 100).item()

With training and evaluation functions, we can finally start training the model!

If the training is done properly, the accuracy should simply reach higher than 0.925:

***Please screenshot the output model accuracy, hand in as YourID_acc_1.png***

In [45]:
for epoch_num in tqdm(range(1, NUM_EPOCH + 1)):
  loss = train(model, dataflow["train"], criterion, optimizer, None)
  acc = evaluate(model, dataflow["test"])
  print(f"epoch {epoch_num}: {acc} | loss: {loss}")

print(f"final accuracy: {acc}")

  0%|          | 0/10 [00:00<?, ?it/s]

train:   0%|          | 0/390 [00:00<?, ?it/s]

eval:   0%|          | 0/78 [00:00<?, ?it/s]

epoch 1: 86.64863586425781 | loss: 0.9129108723157492


train:   0%|          | 0/390 [00:00<?, ?it/s]

...

eval:   0%|          | 0/78 [00:00<?, ?it/s]

epoch 9: 93.03886413574219 | loss: 0.03629232932311984


train:   0%|          | 0/390 [00:00<?, ?it/s]

eval:   0%|          | 0/78 [00:00<?, ?it/s]

epoch 10: 93.05889892578125 | loss: 0.025331564106715797
final accuracy: 93.05889892578125


Save the weight of the model as "model.pt":

In [46]:
# TODO:
# Save the model weight
model_path = "model.pt"
torch.save(model.state_dict(), model_path)

You will find "model.pt" in the current folder.

### Export Model (5%)

We can also save the model weight in [ONNX Format](https://pytorch.org/docs/stable/onnx_torchscript.html):

In [47]:
import torch.onnx

# TODO:
# Specify the input shape
dummy_input = torch.randn(1, 3, 224, 224).cuda()

onnx_path = 'model.onnx'

# TODO:
# Export the model to ONNX format
torch.onnx.export(model, dummy_input, onnx_path, verbose=False)

print(f"Model exported to {onnx_path}")

Model exported to model.onnx


In onnx format, we can observe the model structure using [Netron](https://netron.app/).

***Please download the model structure, hand in as YourID_onnx.png.***

### Inference (10%)

Load the saved model weight:



In [48]:
# TODO:
# Step 1: Get the model structure (mobilenet_v2 and the classifier)
loaded_model = model
# Step 2: Load the model weight from "model.pt".
loaded_model.load_state_dict(torch.load(model_path))
# Step 3: Send the model from cpu to gpu
loaded_model = loaded_model.to("cuda")

<ipython-input-48-2e2193cf1432>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load(model_path))


Run inference with the loaded model weight and check the accuracy

***Please screenshot the output model accuracy, hand in as YourID_acc_2.png***

In [49]:
acc = evaluate(loaded_model, dataflow["test"])
print(f"accuracy: {acc}")

eval:   0%|          | 0/78 [00:00<?, ?it/s]

accuracy: 93.05889892578125


If the accurracy is the same as the accuracy before saved, you have completed PART 1.

Congratulations!

# **Part 2: LLM with torch.compile**

In part 2, we will compare the inference speed of the LLM whether we use torch.compile.

```torch.compile``` is a new feature in PyTorch 2.0.

The following tutorial will help you get to know the usage.

[Introduction to torch.compile](https://pytorch.org/tutorials/intermediate/torch_compile_tutorial.html)

We will choose ```Llama-3.2-1B-Instruct``` as our LLM model.

Make sure you have access to llama before starting Part 2.

https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct

### Loading LLM (20%)

We will first install huggingface and login with your token

In [2]:
!pip install -U "huggingface_hub[cli]"
!huggingface-cli login

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.28.1
    Uninstalling huggingface-hub-0.28.1:
      Successfully uninstalled huggingface-hub-0.28.1

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://hugg

We choose LLaMa 3.2 1B Instruct as our LLM model and load the pretrained model.

Model ID: **"meta-llama/Llama-3.2-1B-Instruct"**


In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# TODO:
# Load the LLaMA 3.2 1B Instruct model
model_id = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

First we need to decide our prompt to feed into LLM and the maximum token length as well.

You can also change the iteration times of testing for the following tests.

In [22]:
# TODO:
# Input prompt
# You can change the prompt whatever you want, e.g. "How to learn a new language?", "What is Edge AI?"

prompt = "What is Edge AI?"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
max_token_length = 1024
iter_times = 8

### Inference with torch.compile (10%)


Let's define a timer function to compare the speed up of ```torch.compile```

In [23]:
def timed(fn):
  start = torch.cuda.Event(enable_timing=True)
  end = torch.cuda.Event(enable_timing=True)
  start.record()
  result = fn()
  end.record()
  torch.cuda.synchronize()
  return result, start.elapsed_time(end) / 1000

After everything is set up, let's start!

We first simply run the inference without ```torch.compile```


In [24]:
original_times = []

# Warm up
with torch.no_grad():
  model.generate(**inputs, max_length=max_token_length, pad_token_id=tokenizer.eos_token_id)

# Timing without torch.compile
for i in range(iter_times):
  with torch.no_grad():
    original_output, original_time = timed(lambda: model.generate(**inputs, max_length=max_token_length, pad_token_id=tokenizer.eos_token_id))
  original_times.append(original_time)
  print(f"Time taken without torch.compile: {original_time} seconds")

# Decode the output
output_text = tokenizer.decode(original_output[0], skip_special_tokens=True)
print(f"Output without torch.compile: {output_text}")

Time taken without torch.compile: 11.413431640625 seconds
Time taken without torch.compile: 9.85898046875 seconds
Time taken without torch.compile: 9.208109375 seconds
Time taken without torch.compile: 7.76295947265625 seconds
Time taken without torch.compile: 9.7442412109375 seconds
Time taken without torch.compile: 13.4151611328125 seconds
Time taken without torch.compile: 7.1352333984375 seconds
Time taken without torch.compile: 9.43986328125 seconds
Output without torch.compile: What is Edge AI? Edge AI refers to Artificial Intelligence (AI) and Machine Learning (ML) that is processed and executed on edge devices, such as smartphones, smart home devices, and other IoT devices, rather than in the cloud. This approach aims to reduce latency, improve real-time processing, and enhance the overall user experience by leveraging the power of AI and ML directly at the edge of the network.

Edge AI involves several key characteristics:

1. **Processing at the edge**: AI and ML algorithms ar

Before using ```torch.compile```, we need to access the model's ```generation_config``` attribute and set the ```cache_implementation``` to "static".

To use ```torch.compile```, we need to call ```torch.compile``` on the model to compile the forward pass with the static kv-cache.

Reference: https://huggingface.co/docs/transformers/llm_optims?static-kv=basic+usage%3A+generation_config

In [25]:
compile_times = []
# Remind that whenever you use torch.compile, you need to use torch._dynamo.reset() to clear all compilation caches and restores the system to its initial state.
import torch._dynamo
torch._dynamo.reset()
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # To prevent long warnings :)

# TODO:
# Compile the model
model.generation_config.cache_implementation = "static"
compiled_model = torch.compile(model, mode="max-autotune", fullgraph=True)

# Warm-up
with torch.no_grad():
  compiled_model.generate(**inputs, max_length=max_token_length, pad_token_id=tokenizer.eos_token_id)

# Timing with torch.compile
for i in range(iter_times):
  with torch.no_grad():
    compile_output, compile_time = timed(lambda: compiled_model.generate(**inputs, max_length=max_token_length, pad_token_id=tokenizer.eos_token_id))
  compile_times.append(compile_time)
  print(f"Time taken with torch.compile: {compile_time} seconds")

# Decode output
output_text = tokenizer.decode(compile_output[0], skip_special_tokens=True)
print(f"\nOutput with torch.compile: {output_text}")

Time taken with torch.compile: 6.23961962890625 seconds
Time taken with torch.compile: 7.5533349609375 seconds
Time taken with torch.compile: 8.10957080078125 seconds
Time taken with torch.compile: 5.17078564453125 seconds
Time taken with torch.compile: 6.46395947265625 seconds
Time taken with torch.compile: 6.20576904296875 seconds
Time taken with torch.compile: 7.0231943359375 seconds
Time taken with torch.compile: 5.73454736328125 seconds

Output with torch.compile: What is Edge AI? Edge AI refers to artificial intelligence (AI) that is processed and analyzed on the edge, i.e., on devices or systems that are closer to the data source, such as smartphones, smart home devices, or autonomous vehicles. The Edge AI system is designed to be faster, more efficient, and more reliable than traditional cloud-based AI systems.

Edge AI has several key characteristics:

1. **Faster processing**: Edge AI systems process data closer to where it is generated, reducing latency and enabling real-tim

We can easily observe that after the first inference, the inference time drops a lot!

Below code can tell you how much faster did ```torch.compile``` did.

***Please screenshot the inference time and speedup below, hand in as YourID_speedup.png***

In [27]:
import numpy as np
original_med = np.median(original_times)
compile_med = np.median(compile_times)
speedup = original_med / compile_med
print(f"Original median: {original_med},\nCompile median: {compile_med},\nSpeedup: {speedup}x")

Original median: 9.59205224609375,
Compile median: 6.35178955078125,
Speedup: 1.51013382439819x


You've finished part 2.

Congratulations!